# Step 0. Parameter Setting

In [ ]:
import dtnmonitor_v2 as dtn
import os,sys


### P1. SELECT THE MONITOR NETWORK INTERFACE
## used in Step 1,2,3 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
#dtn.interface = 'eth0.2038' 
dtn.interface = 'ens1f0.3061'
#dtn.interface = 'eth1'
#dtn.interface = 'all'


### P2. SELECT THE INPUT FILE , 
## used in Step 1 : file IO read file



file_read_nuttcp=[
"/data/disk0/sc17/100gtest",
"/data/disk1/sc17/100gtest",
"/data/disk2/sc17/100gtest",
"/data/disk3/sc17/100gtest",
"/data/disk4/sc17/100gtest",
"/data/disk5/sc17/100gtest",
"/data/disk6/sc17/100gtest",
"/data/disk7/sc17/100gtest",
    # "/foo/bar/filennn"
     ]

file2_read_nuttcp=[
"/data/disk0/sc17/fftest",
"/data/disk1/sc17/fftest",
"/data/disk2/sc17/fftest",
"/data/disk3/sc17/fftest",
"/data/disk4/sc17/fftest",
"/data/disk5/sc17/fftest",
"/data/disk6/sc17/fftest",
"/data/disk7/sc17/fftest",
    # "/foo/bar/filennn"
     ]






### P3. NUTTCP SERVER IP 
## used in Step 2,3 : input nuttcp server's ip
nuttcp_server="192.168.61.57"

### P4. NUTTCP THREADS
## used in Step 2,3 :input threads number
count = 8

## run fio and check status
### select graph_mode [0|1] 
#graph_mode=0  # all, 
#graph_mode=1  # 100pt
graph_mode=0  

### Show Disk and Interface
import netifaces
import os
list=netifaces.interfaces()
print("==========================================================")
print('{:40s} {:20s} '.format("Network Interface","IP"))
print("==========================================================")
for inf in list:
    #print(inf)
    ff=netifaces.ifaddresses(inf)
    #print(ff)
    try:
        addr=ff[netifaces.AF_INET][0]['addr']
    except:
        continue
    print('{:40s} {:20s} '.format(inf,addr)) 
import subprocess
import re
df = subprocess.Popen(["df","-h"], stdout=subprocess.PIPE)
output = df.communicate()[0]
dfs = output.decode("utf8").split("\n")
dfs.pop(0)
print("==========================================================")
print('{:35s} {:7s}  {:7s} {:7s}'.\
      format("Disk mountpoint","size","avail","used%"))
print("==========================================================")
for i in dfs:
    try:
        xx = re.sub(" +"," ",i).split(" ")
        if len(xx) == 6:
            device, size, used, available, percent, mountpoint = xx
            print('{:35s} {:7s}  {:7s} {:7s}'\
                  .format(mountpoint,size,available,percent))
    except:
        continue
import getpass
user=getpass.getuser()
def exec_print(command):
    process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
    for line in iter(process.stdout.readline, b''):
        # system print
        sys.stdout.write(line)   

In [ ]:
print("==========================================================")
print('{:60s} '.format("Initial checking"))
print("==========================================================")

command="ps aux |grep 'python3 '|wc -l"
process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
ret=process.stdout.read().decode('utf8')



if int(ret) > 2: 
    print("System check ..... [ok]")
    print("Error!! Some process are running, please check first!!!")
    exec_print("ps -u "+ user +" ux |grep 'python3 '")
else : 
    print("System check ..... [ok]")
    print('Monitoring Network Interface => {:15s} '.format(dtn.interface))

# Step 1. Test network transfer by disk

In [ ]:
for i in range(count):
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5300"+str(i)+" -p 5310"+str(i)+ " "+ nuttcp_server +" < "+ file_read_nuttcp[i] +" & "
    print(command)
    #os.system(command)
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5320"+str(i)+" -p 5330"+str(i)+ " "+ nuttcp_server +" < "+ file2_read_nuttcp[i] +" & "
    print(command)
    #os.system(command)
 


#dtn.exec_command("sleep 300",graph_mode)

In [ ]:
for i in range(count):
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5300"+str(i)+" -p 5310"+str(i)+ " "+ nuttcp_server +" < "+ file_read_nuttcp[i] +" & "
    #print(command)
    os.system(command)
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5320"+str(i)+" -p 5330"+str(i)+ " "+ nuttcp_server +" < "+ file2_read_nuttcp[i] +" & "
    os.system(command)
 


dtn.exec_command("sleep 300",graph_mode)